In [1]:
import re
import sys
import json
import gzip
import logging
from functools import lru_cache
from multiprocessing import Pool

import pymorphy2
import numpy as np
import pandas as pd
from annoy import AnnoyIndex
from gensim.models.word2vec import Word2Vec
import pickle as pkl


In [2]:
import pickle as pkl

In [3]:
Product_dict = pkl.load(open('Product_dict.pkl', 'rb'))

In [4]:
Product_dict

{'168308': '(197312) Пакет-майка 25см х 45см 906',
 '134832': '(62448) Перекись водорода р-р наружн. 3% фл.полимерн. 100мл 404',
 '101384': '(72183) Салициловая кислота р-р спирт 2% фл 40мл N1 404',
 '168570': '(197309) Пакет 28см х 50см 906',
 '146960': '(111023) Пакет "Аптека Озерки" 28 х 35см 906',
 '128700': '(51806) Ранитидин тб 150мг уп N20 595',
 '52539': '(19416) Платки носовые "Зева Кидс" №10 721',
 '94270': '(66684) Пакет майка "Благодарим за покупку" (28+14) х 43см №1 -1',
 '77426': '(5605) Иглы Микро-Файн д/шприц-ручки 30G 0,3*8мм N100 759',
 '92475': '(114214) Дорзопт капли глазные 2% 5мл. фл.-кап. N3 575',
 '158643': '(181230) PL Шприц одноразовый 3-комп. 3мл №1 762',
 '158600': '(181232) PL Шприц одноразовый 3-комп. 5мл №1 762',
 '106230': '(43344) Аллапинин тб 25мг N30 738',
 '110629': '(57733) Корвалол капли д/приема внутрь 25мл 738',
 '47251': '(121286) PL Контейнер д/биопроб стер. 60мл с крыш и ложкой в инд.уп 711',
 '39811': '(116543) PL Салфетка антисептическая спи

In [5]:
type(Product_dict)

dict

In [6]:
data=pd.read_csv('data11.csv',nrows=100000)

In [7]:
data

,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455,168308,(197312) Пакет-майка 25см х 45см,906,205,-1,"1,00"
1,2018-12-07,198287,279,134832,(62448) Перекись водорода р-р наружн. 3% фл.по...,404,93,-1,"1,00"
2,2018-12-07,2418385,848,101384,(72183) Салициловая кислота р-р спирт 2% фл 40...,404,93,-1,"1,00"
3,2018-12-07,1285774,1511,168570,(197309) Пакет 28см х 50см,906,205,-1,"1,00"
4,2018-12-07,1810323,1501,168319,(197310) Пакет 30см х 60см,906,205,-1,"1,00"
5,2018-12-07,2435604,66,37505,(53208) Кеппра таб.п.п.о.500мг №30,641,146,-1,"1,00"
6,2018-12-07,1697532,1504,168570,(197309) Пакет 28см х 50см,906,205,-1,"1,00"
7,2018-12-07,1779333,1455,168308,(197312) Пакет-майка 25см х 45см,906,205,-1,"1,00"
8,2018-12-07,507339,275,134832,(62448) Перекись водорода р-р наружн. 3% фл.по...,404,93,-1,"1,00"
9,2018-12-07,1533558,1462,168308,(197312) Пакет-майка 25см х 45см,906,205,-1,"1,00"


In [8]:
data=data.applymap(str)


In [9]:
type(data['product_sub_category_id'][0])

str

In [10]:
data1=data.drop(columns=['contact_id','shop_id','product_category_id'])

In [11]:
data1['id']=data['contact_id']+' '+data['shop_id']+' ' +data['product_category_id']

In [12]:
data1

,sale_date_date,product_id,name,product_sub_category_id,brand_id,quantity,id
0,2018-12-07,168308,(197312) Пакет-майка 25см х 45см,906,-1,"1,00",1260627 1455 205
1,2018-12-07,134832,(62448) Перекись водорода р-р наружн. 3% фл.по...,404,-1,"1,00",198287 279 93
2,2018-12-07,101384,(72183) Салициловая кислота р-р спирт 2% фл 40...,404,-1,"1,00",2418385 848 93
3,2018-12-07,168570,(197309) Пакет 28см х 50см,906,-1,"1,00",1285774 1511 205
4,2018-12-07,168319,(197310) Пакет 30см х 60см,906,-1,"1,00",1810323 1501 205
5,2018-12-07,37505,(53208) Кеппра таб.п.п.о.500мг №30,641,-1,"1,00",2435604 66 146
6,2018-12-07,168570,(197309) Пакет 28см х 50см,906,-1,"1,00",1697532 1504 205
7,2018-12-07,168308,(197312) Пакет-майка 25см х 45см,906,-1,"1,00",1779333 1455 205
8,2018-12-07,134832,(62448) Перекись водорода р-р наружн. 3% фл.по...,404,-1,"1,00",507339 275 93
9,2018-12-07,168308,(197312) Пакет-майка 25см х 45см,906,-1,"1,00",1533558 1462 205


In [13]:
sentences = data1.id.str.split()


In [14]:
type(sentences[0][1])

str

In [15]:
sentences

0        [1260627, 1455, 205]
1           [198287, 279, 93]
2          [2418385, 848, 93]
3        [1285774, 1511, 205]
4        [1810323, 1501, 205]
5          [2435604, 66, 146]
6        [1697532, 1504, 205]
7        [1779333, 1455, 205]
8           [507339, 275, 93]
9        [1533558, 1462, 205]
10       [2030547, 1146, 205]
11       [1940123, 1495, 205]
12         [785614, 859, 194]
13       [1301620, 1471, 205]
14       [1455298, 1495, 205]
15       [2520496, 1004, 205]
16       [1300685, 1918, 205]
17         [514490, 350, 135]
18       [1493592, 1516, 205]
19        [2109210, 324, 167]
20         [1993783, 303, -1]
21        [2742478, 105, 176]
22         [2480204, 303, -1]
23          [87961, 596, 129]
24       [2650451, 1496, 205]
25        [2650603, 840, 176]
26        [2605787, 457, 176]
27         [364724, 268, 170]
28         [364724, 268, 170]
29       [1347624, 1495, 146]
                 ...         
99970     [1205575, 144, 140]
99971      [1075439, 946, 90]
99972     

In [17]:
model = Word2Vec(sentences, size=100, workers=4, iter=10,window=3,sg=1,min_count=1)
model.save('./hackaton.w2v_gensim100')

In [18]:
model1 = Word2Vec.load('./hackaton.w2v_gensim100')

In [19]:
model.similar_by_vector(model1['1260627'] )

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """Entry point for launching an IPython kernel.


[('1260627', 1.0),
 ('1259082', 0.9979445934295654),
 ('2531581', 0.9977498054504395),
 ('1356615', 0.9976977109909058),
 ('118', 0.9976115822792053),
 ('2737149', 0.9976006746292114),
 ('1605379', 0.9975553154945374),
 ('1658953', 0.9975452423095703),
 ('1360114', 0.997520923614502),
 ('2631142', 0.9974320530891418)]

In [20]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))
word_to_idx = {} # для преобразования текста к индексам - скармливание модели.
counter = 0
embed_matrix = []

for i in w2v:
    word_to_idx[i] = counter
    embed_matrix.append(w2v[i])
embed_matrix = np.vstack(embed_matrix)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [21]:
data_storage = {i[1]['product_id']:i[1]['id'] for i in data1.iterrows()}

In [22]:
data_storage

{'168308': '2737300 1455 205',
 '134832': '2737717 895 93',
 '101384': '965024 323 93',
 '168570': '745320 52 205',
 '168319': '2739426 1503 205',
 '37505': '2435604 66 146',
 '146960': '2182394 109 205',
 '158662': '2737971 978 194',
 '128700': '905706 318 135',
 '52539': '380093 268 167',
 '94270': '1156074 302 -1',
 '77426': '2657958 298 176',
 '92475': '2690571 268 129',
 '158643': '2106953 2166 176',
 '158600': '2737454 144 176',
 '106230': '2195822 1144 170',
 '110629': '2496400 917 170',
 '89410': '1347624 1495 146',
 '106860': '2361160 852 165',
 '47251': '362487 145 165',
 '154704': '2196478 323 147',
 '39811': '571857 276 93',
 '143709': '2310157 1432 128',
 '25965': '2721183 2114 93',
 '130823': '2610324 898 166',
 '158645': '727046 2063 176',
 '54512': '62343 351 162',
 '79879': '182435 349 92',
 '100667': '2304258 455 101',
 '87332': '1988399 992 129',
 '34104': '739101 242 142',
 '21249': '2685941 997 140',
 '158659': '2736875 1994 176',
 '85136': '697304 271 154',
 '7620

In [23]:
data_storage_norm = {}
for i in (data_storage):
    text = data_storage[i]
    vec = np.zeros(100)
    for word in text.split(' '):
        if word in model :
            vec+=model[word] 

    data_storage_norm[i] = vec

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [24]:
data_storage_norm

{'168308': array([-0.92709858,  0.39354312, -0.82811213,  1.13943635,  0.38880065,
         0.95068049,  0.46120786,  0.96843199,  0.11087206,  0.36601346,
        -1.13629104, -0.69841772,  0.59331869,  0.28027557,  0.75286062,
        -1.6458348 ,  0.56319671, -1.57548621, -0.21978035,  1.2235982 ,
        -0.01735197, -0.62504921, -0.34345248, -0.8701279 ,  0.8940423 ,
         1.33048755, -0.9451412 ,  1.2549478 ,  0.4964987 , -1.11710887,
        -1.58036795, -0.44437395, -0.65191103, -0.65033821,  0.4671346 ,
         0.11886994, -0.41858866,  0.43839423, -0.89273354, -1.75007431,
         2.02457511, -0.27110936, -0.50632278, -0.40350688,  2.1746709 ,
        -0.64605087,  0.14041525, -0.43101158, -0.34577853, -1.04366168,
        -0.64283381,  0.00378035, -0.24032317, -0.04475999,  1.86945105,
        -0.15439974, -2.43820496, -0.89389021, -2.09221861, -0.87698975,
        -0.5257578 , -0.98300105,  0.26474543,  0.10791433, -0.4532286 ,
        -0.59727722, -0.06223158, -0.0914

In [25]:
len(data_storage_norm)

8762

In [26]:
from annoy import AnnoyIndex

NUM_TREES = 100
VEC_SIZE_EMB = 100

counter = 0
map_id_2_prod_hash = {}
index_title_emb = AnnoyIndex(VEC_SIZE_EMB)

print("Build annoy base")
for prod_hash in data_storage_norm:
    title_vec = data_storage_norm[prod_hash] # Вытаскиваем вектор
    
    index_title_emb.add_item(counter, title_vec) # Кладем в анной
    
    map_id_2_prod_hash[counter] = prod_hash # сохраняем мапу - (id в анное -> продукт_id)
    
    counter += 1
    if counter % 10000 == 1:
        print("computed for %d" % counter)
        
index_title_emb.build(NUM_TREES)
print("builded")

Build annoy base
computed for 1
builded


In [27]:
index_title_emb.save('./hackaton_annoy_100')
pkl.dump(map_id_2_prod_hash, open('hackaton_map_id_to_hash_products.dict100', 'wb'))

In [31]:
index_img_emb1= AnnoyIndex(VEC_SIZE_EMB)
index_img_emb1.load('./hackaton_annoy_100')
map_id_hashimg1 = pkl.load(open('hackaton_map_id_to_hash_products.dict100', 'rb'))

In [30]:
import itertools

In [36]:
title_id = list(data_storage_norm.keys())[np.random.randint(1000)]

print('Запрос', data_storage[title_id])
print('векторок', data_storage_norm[title_id][:])

annoy_res = list(index_img_emb1.get_nns_by_vector(data_storage_norm[title_id], 13, include_distances=True))

print('\n\nСоседи:')

for annoy_id, annoy_sim in itertools.islice(zip(*annoy_res), 13):
    image_id = map_id_hashimg1[annoy_id]
    print(data_storage[image_id], 1 - annoy_sim ** 2 / 2)
    print('___' * 13)

Запрос 2499560 1009 146
векторок [-0.68179746 -0.09378441 -0.82699299  0.6344634   0.43913915  0.93515541
 -0.01478962  0.62181926 -0.15408566  0.46967403 -0.98606715 -0.86376561
  0.75471148 -0.01975887  0.36068248 -1.47865208  0.08641641 -0.73660295
  0.01513382  0.63945591  0.2189192  -0.12658727 -0.62746582 -0.73301036
  0.52298732  0.85200498 -0.24678497  0.6007213   0.52506951 -0.64735668
 -0.92412917  0.03877388 -0.5639639  -0.15348512  0.49154585  0.30203571
 -0.21358446  0.48555911 -0.4774087  -0.83023581  1.55978341 -0.46795008
 -0.40374038 -0.07726322  1.41252463 -0.37010995  0.27136392 -0.40686188
 -0.28131486 -0.57068449 -0.57865285  0.45699339  0.084482   -0.13974633
  1.47842891 -0.04056303 -0.97920383 -0.43022379 -1.1027767  -0.97503191
 -0.20781947 -0.77508437 -0.14242346 -0.17923357  0.05772134 -0.04792365
 -0.09813858 -0.25975212  0.61469838  0.08256332  0.07933253  0.16174072
 -0.01911515  0.53170563  0.22547125  0.21338277 -0.16558294  0.36710648
  0.34009617 -0.76

In [37]:
vector='2418385 848 93'

vec = np.zeros(100)
for word in vector.split(' '):
    if word in model:
        vec+=model[word] 

data_storage_norm1 = vec

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [38]:
Product_dict.keys()

dict_keys(['168308', '134832', '101384', '168570', '146960', '128700', '52539', '94270', '77426', '92475', '158643', '158600', '106230', '110629', '47251', '39811', '143709', '25965', '158645', '79879', '100667', '87332', '21249', '158659', '76206', '114366', '34531', '27063', '20236', '99525', '116091', '97117', '160521', '94899', '67922', '98991', '90247', '28232', '60404', '19578', '124839', '42409', '77808', '60228', '110342', '124535', '79735', '68982', '135018', '141455', '148629', '55357', '135427', '19510', '66818', '141996', '161562', '155688', '155686', '162991', '87663', '145608', '108808', '127911', '141757', '153569', '115144', '32087', '71618', '114395', '65122', '357926', '94047', '145106', '140683', '110112', '79642', '215298', '75266', '122488', '98400', '139676', '155555', '155560', '129205', '44867', '349737', '137039', '23345', '66533', '22470', '37982', '129521', '115228', '137031', '153754', '113441', '117000', '23289', '77662', '124061', '83462', '66485', '233213

In [39]:
prod=pd.DataFrame.from_dict(Product_dict,orient='index')

In [40]:
prod[0]=prod[0].str.replace(r"\(.*\)","")
prod[0]=prod[0].str.replace('0',"")
prod[0]=prod[0].str.replace('1',"")
prod[0]=prod[0].str.replace('2',"")
prod[0]=prod[0].str.replace('3',"")
prod[0]=prod[0].str.replace('4',"")
prod[0]=prod[0].str.replace('5',"")
prod[0]=prod[0].str.replace('6',"")
prod[0]=prod[0].str.replace('7',"")
prod[0]=prod[0].str.replace('8',"")
prod[0]=prod[0].str.replace('9',"")


In [41]:
prod[0]

168308                                 Пакет-майка см х см 
134832     Перекись водорода р-р наружн. % фл.полимерн. мл 
101384             Салициловая кислота р-р спирт % фл мл N 
168570                                       Пакет см х см 
146960                         Пакет "Аптека Озерки"  х см 
128700                                Ранитидин тб мг уп N 
52539                         Платки носовые "Зева Кидс" № 
94270                                              х см № -
77426               Иглы Микро-Файн д/шприц-ручки G ,*мм N 
92475               Дорзопт капли глазные % мл. фл.-кап. N 
158643                    PL Шприц одноразовый -комп. мл № 
158600                    PL Шприц одноразовый -комп. мл № 
106230                                   Аллапинин тб мг N 
110629                   Корвалол капли д/приема внутрь мл 
47251      PL Контейнер д/биопроб стер. мл с крыш и ложк...
39811          PL Салфетка антисептическая спиртовая хсм № 
143709                  PL вода питьевая

In [42]:
annoy_res = list(index_title_emb.get_nns_by_vector(data_storage_norm1, 13, include_distances=True,search_k=10000))

print('\n\nСоседи:')

for annoy_id, annoy_sim in itertools.islice(zip(*annoy_res), 13):
    image_id = map_id_2_prod_hash[annoy_id]
   # print(image_id)
    #print(data_storage[image_id], 1 - annoy_sim ** 2 / 2)
    #print(prod.loc[prod['0'] == str(image_id)])
    print(prod[0][image_id])
    #print('___' * 13)



Соседи:
 PL Салфетка антисептическая спиртовая ,х,см № 
 Холисал гель стоматологический туба г 
 Компид Интенсив Пластырь от сухих мозолей на ногах, средний шт. уп. № 
 Компид Пластырь от сухих враст. мозол. на ногах, средний шт. уп. № 
 ЭВО крем-мыло д/интимной гигиены мл 
 Малавит крем-гель д/тела мл 
 Фарингосепт таб. д/рассас. Корица № 
 Аммиак р-р % мл N 
 Фурацилин р-р спирт фл мл N 
 Бальзамический линимент по Вишневскому г N 
 Мозоль Ка средство от мозолей и натоптышей фл. мл. 
 Ципролет А табл. п.п.о. +мг № 
 Салициловая кислота р-р спирт % фл мл N 
